In [0]:
import requests
import pytz
import csv
import time
from datetime import datetime, timedelta
from pyspark.sql import Row  

In [0]:
API_KEY = 'UiuE73uNUuG9AwnT1pRMOLayYw6ZjX4z'
BASE_URL = 'https://aeroapi.flightaware.com/aeroapi'

headers = {
    'x-apikey': API_KEY
}

In [0]:
# Define Malaysia's timezone
malaysia_tz = pytz.timezone('Asia/Kuala_Lumpur')

# Get the current time in Malaysia
malaysia_time = datetime.now(malaysia_tz)

# Subtract one and two hours
malaysia_time_minus_one_hour = malaysia_time - timedelta(hours=4)
malaysia_time_minus_two_hour = malaysia_time - timedelta(hours=5)

# Format the time in the desired format
start_time_myt = '2024-09-01T21:00:00'
end_time_myt = '2024-09-02T00:00:00'

airport_code = 'WMKK'

print(start_time_myt)
print(end_time_myt)

2024-09-01T21:00:00
2024-09-02T00:00:00


TRANSFORMATION DEPARTURE

In [0]:
time.sleep(65)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import Row

def sanitize_filename(filename):
    # Replace problematic characters with underscores
    return filename.replace(':', '_')

def convert_myt_to_utc(myt_time_str):
    myt_time = datetime.strptime(myt_time_str, '%Y-%m-%dT%H:%M:%S')
    utc_time = myt_time - timedelta(hours=8)
    return utc_time.isoformat()

def convert_utc_to_myt(utc_time_str):
    # Handle missing or invalid date strings
    if utc_time_str and utc_time_str != 'N/A':
        try:
            utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%SZ')
            myt_time = utc_time + timedelta(hours=8)
            return myt_time.strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            return 'Invalid Date'
    else:
        return 'N/A'

def get_recent_departures(airport_code, start_time_myt, end_time_myt):
    start_time_iso = convert_myt_to_utc(start_time_myt) + 'Z'
    end_time_iso = convert_myt_to_utc(end_time_myt) + 'Z'
    
    endpoint = f'{BASE_URL}/airports/{airport_code}/flights/departures'
    params = {
        'start': start_time_iso,
        'end': end_time_iso,
        'max_pages':10
    }
    
    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

def convert_to_dataframe_departures(spark, data):
    # Define the schema explicitly
    schema = StructType([
        StructField("flight_id", StringType(), True),
        StructField("flight_number", StringType(), True),
        StructField("aircraft_type", StringType(), True),
        StructField("scheduled_departure_myt", StringType(), True),
        StructField("actual_departure_myt", StringType(), True),
        StructField("origin", StringType(), True),
        StructField("destination", StringType(), True),
        StructField("gate_destination", StringType(), True)
    ])
    
    # Create a list of Row objects with safe checks for None
    rows = [
        Row(
            flight_id=flight.get('ident', 'N/A'),
            flight_number=flight.get('flight_number', 'N/A'),
            aircraft_type=flight.get('aircraft_type', 'N/A'),
            scheduled_departure_myt=convert_utc_to_myt(flight.get('scheduled_off', 'N/A')),
            actual_departure_myt=convert_utc_to_myt(flight.get('actual_off', 'N/A')),
            origin=flight.get('origin', {}).get('name', 'N/A') if flight.get('origin') else 'N/A',
            destination=flight.get('destination', {}).get('name', 'N/A') if flight.get('destination') else 'N/A',
            gate_destination=flight.get('gate_destination', 'N/A')
        )
        for flight in data.get('departures', [])
    ]
    
    # Convert the list of Rows into a DataFrame with the predefined schema
    df = spark.createDataFrame(rows, schema=schema)
    return df

# Fetch recent departures
recent_departures = get_recent_departures(airport_code, start_time_myt, end_time_myt)

# Check if the data is in JSON format
if isinstance(recent_departures, dict):
    df_departures = convert_to_dataframe_departures(spark, recent_departures)
    df_departures.show()  # Display the DataFrame
else:
    print("Error fetching data:", recent_departures)


+---------+-------------+-------------+-----------------------+--------------------+------------------+--------------------+----------------+
|flight_id|flight_number|aircraft_type|scheduled_departure_myt|actual_departure_myt|            origin|         destination|gate_destination|
+---------+-------------+-------------+-----------------------+--------------------+------------------+--------------------+----------------+
|   KLM810|          810|        B789 |    2024-09-01 23:55:00| 2024-09-01 23:58:46|Kuala Lumpur Int'l|  Amsterdam Schiphol|              F7|
|   AXM103|          103|        A320 |    2024-09-01 21:40:00| 2024-09-01 23:55:02|Kuala Lumpur Int'l|Kunming Changshui...|            NULL|
|   ETH639|          639|        B788 |    2024-09-01 23:30:00| 2024-09-01 23:51:31|Kuala Lumpur Int'l|    Singapore Changi|             E26|
|  AXM5120|         5120|        A320 |    2024-09-01 23:50:00| 2024-09-01 23:49:10|Kuala Lumpur Int'l| Kota Kinabalu Int'l|            NULL|
|  MAS

SAVE FILES DEPARTURE

In [0]:
# Sanitize the start time for use in filenames and paths
sanitized_start_time = sanitize_filename(start_time_myt)

# Define the folder path directly
folder_path_departures = '/mnt/raw/departures/'

# Coalesce the DataFrame to a single partition
df_departures_coalesced = df_departures.coalesce(1)

# Write the coalesced DataFrame to a temporary folder
temp_folder_path_departures = f"{folder_path_departures}temp_departures_{sanitized_start_time}/"
df_departures_coalesced.write.mode('overwrite').option('header', 'true').csv(temp_folder_path_departures)

# List the files in the temporary folder after writing the CSV file
files_departures = dbutils.fs.ls(temp_folder_path_departures)

# Filter out the actual CSV file (ignore _SUCCESS and other metadata files)
csv_temp_file = [f.path for f in files_departures if f.path.endswith('.csv')][0]

# Define the final file path for the CSV file
final_csv_file_path_departures = f"{folder_path_departures}departures_{sanitized_start_time}.csv"

# Move the CSV file to the final destination
dbutils.fs.mv(csv_temp_file, final_csv_file_path_departures)

# Remove the temporary folder and its contents (_committed_, _started_, _SUCCESS)
dbutils.fs.rm(temp_folder_path_departures, True)


True

In [0]:
# Sanitize the start time for use in filenames and paths
#sanitized_start_time = sanitize_filename(start_time_myt)

# Define the folder path before writing the file
#folder_path_departures = f'/mnt/raw/departures/departures_{sanitized_start_time}/'

# Coalesce the DataFrame to a single partition
#df_departures_coalesced = df_departures.coalesce(1)

# Write the coalesced DataFrame to a CSV file in the specified folder
#df_departures_coalesced.write.mode('overwrite').option('header', 'true').csv(folder_path_departures)

# List the files in the directory after writing the CSV file
#files_departures = dbutils.fs.ls(folder_path_departures)

# Filter out the actual CSV file (ignore _SUCCESS and other metadata files)
#csv_file = [f.path for f in files_departures if f.path.endswith('.csv')][0]

# Correct the file path for renaming
#corrected_file_path_departures = f"{folder_path_departures}departures_{sanitized_start_time}.csv"

# Rename the file
#dbutils.fs.mv(csv_file, corrected_file_path_departures)


TRANSFORMATION ARRIVAL

In [0]:
time.sleep(65)

In [0]:
def get_recent_arrivals(airport_code, start_time_myt, end_time_myt):
    end_time_iso = convert_myt_to_utc(end_time_myt) + 'Z'
    start_time_iso = convert_myt_to_utc(start_time_myt) + 'Z'
    
    endpoint = f'{BASE_URL}/airports/{airport_code}/flights/arrivals'
    params = {
        'start': start_time_iso,
        'end': end_time_iso,
        'max_pages':10
    }
    
    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

def convert_to_dataframe_arrivals(spark, data):
    # Define the schema explicitly for arrivals
    schema = StructType([
        StructField("flight_id", StringType(), True),
        StructField("flight_number", StringType(), True),
        StructField("aircraft_type", StringType(), True),
        StructField("scheduled_arrival_myt", StringType(), True),
        StructField("actual_arrival_myt", StringType(), True),
        StructField("origin", StringType(), True),
        StructField("destination", StringType(), True),
        StructField("gate_destination", StringType(), True)
    ])
    
    # Create a list of Row objects
    rows = [
        Row(
            flight_id=flight.get('ident', 'N/A'),
            flight_number=flight.get('flight_number', 'N/A'),
            aircraft_type=flight.get('aircraft_type', 'N/A'),
            scheduled_arrival_myt=convert_utc_to_myt(flight.get('scheduled_on', 'N/A')),
            actual_arrival_myt=convert_utc_to_myt(flight.get('actual_on', 'N/A')),
            origin=flight.get('origin', {}).get('name', 'N/A'),
            destination=flight.get('destination', {}).get('name', 'N/A'),
            gate_destination=flight.get('gate_destination', 'N/A')
        )
        for flight in data.get('arrivals', [])
    ]
    
    # Convert the list of Rows into a DataFrame with the predefined schema
    df = spark.createDataFrame(rows, schema=schema)
    return df

# Fetch recent arrivals
recent_arrivals = get_recent_arrivals(airport_code, start_time_myt, end_time_myt)

# Check if the data is in JSON format
if isinstance(recent_arrivals, dict):
    df_arrivals = convert_to_dataframe_arrivals(spark, recent_arrivals)
    df_arrivals.show()  # Display the DataFrame
else:
    print("Error fetching data:", recent_arrivals)

+---------+-------------+-------------+---------------------+-------------------+--------------------+------------------+----------------+
|flight_id|flight_number|aircraft_type|scheduled_arrival_myt| actual_arrival_myt|              origin|       destination|gate_destination|
+---------+-------------+-------------+---------------------+-------------------+--------------------+------------------+----------------+
|  AXM6307|         6307|        A320 |  2024-09-01 22:15:00|2024-09-01 23:56:43|      Langkawi Int'l|Kuala Lumpur Int'l|            NULL|
|  MXD1007|         1007|        B738 |  2024-09-02 00:00:00|2024-09-01 23:54:24| Kota Kinabalu Int'l|Kuala Lumpur Int'l|            NULL|
|   CXA823|          823|        B738 |  2024-09-01 23:38:00|2024-09-01 23:50:57|  Xiamen Gaoqi Int'l|Kuala Lumpur Int'l|             C11|
|   MXD623|          623|        B38M |  2024-09-01 22:57:00|2024-09-01 23:46:46|              Dayong|Kuala Lumpur Int'l|            NULL|
|   MXD152|          152|  

SAVE FILES ARRIVALS

In [0]:
# Sanitize the start time for use in filenames and paths
sanitized_start_time = sanitize_filename(start_time_myt)

# Define the folder path directly
folder_path_arrivals = '/mnt/raw/arrivals/'

# Coalesce the DataFrame to a single partition
df_arrivals_coalesced = df_arrivals.coalesce(1)

# Write the coalesced DataFrame to a temporary folder
temp_folder_path_arrivals = f"{folder_path_arrivals}temp_arrivals_{sanitized_start_time}/"
df_arrivals_coalesced.write.mode('overwrite').option('header', 'true').csv(temp_folder_path_arrivals)

# List the files in the temporary folder after writing the CSV file
files_arrivals = dbutils.fs.ls(temp_folder_path_arrivals)

# Filter out the actual CSV file (ignore _SUCCESS and other metadata files)
csv_temp_file = [f.path for f in files_arrivals if f.path.endswith('.csv')][0]

# Define the final file path for the CSV file
final_csv_file_path_arrivals = f"{folder_path_arrivals}arrivals_{sanitized_start_time}.csv"

# Move the CSV file to the final destination
dbutils.fs.mv(csv_temp_file, final_csv_file_path_arrivals)

# Remove the temporary folder and its contents (_committed_, _started_, _SUCCESS)
dbutils.fs.rm(temp_folder_path_arrivals, True)


True

In [0]:
# Sanitize the start time for use in filenames and paths
#sanitized_start_time = sanitize_filename(start_time_myt)

# Define the folder path before writing the file
#folder_path_arrivals = f'/mnt/raw/arrivals/arrivals_{sanitized_start_time}/'

# Coalesce the DataFrame to a single partition
#df_arrivals_coalesced = df_arrivals.coalesce(1)

# Write the coalesced DataFrame to a CSV file in the specified folder (corrected to folder_path_arrivals)
#df_arrivals_coalesced.write.mode('overwrite').option('header', 'true').csv(folder_path_arrivals)

# List the files in the directory after writing the CSV file
#files_arrivals = dbutils.fs.ls(folder_path_arrivals)

# Filter out the actual CSV file (ignore _SUCCESS and other metadata files)
#csv_file_arrivals = [f.path for f in files_arrivals if f.path.endswith('.csv')][0]

# Correct the file path for renaming
#corrected_file_path_arrivals = f"{folder_path_arrivals}arrivals_{sanitized_start_time}.csv"

# Rename the file
#dbutils.fs.mv(csv_file_arrivals, corrected_file_path_arrivals)


In [0]:
departure_row = df_departures.count()
print(departure_row)
arrivals_row = df_arrivals.count()
print(arrivals_row)

82
68
